In [1]:
import pandas as pd 
from datetime import datetime
import numpy as np

# Prep data

In [2]:
df =pd.read_csv('synthetic_honeynet_logs.csv')

In [3]:
df.head()

,timestamp,log_type,client_ip,event
0,2025-03-29T12:19:00Z,auth,192.168.1.11,Accepted password for root from 192.168.1.11 p...
1,2025-03-29T10:46:00Z,access,192.168.1.11,GET /config.php HTTP/1.1 - 403
2,2025-03-29T12:13:00Z,cron,203.0.113.5,CRON job started: wget malicious.sh
3,2025-03-29T12:38:00Z,cron,203.0.113.5,CRON executed reverse_shell.py
4,2025-03-29T12:38:00Z,access,192.168.1.10,GET /admin HTTP/1.1 - 403


In [4]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [5]:
df.head()

,timestamp,log_type,client_ip,event
0,2025-03-29 12:19:00+00:00,auth,192.168.1.11,Accepted password for root from 192.168.1.11 p...
1,2025-03-29 10:46:00+00:00,access,192.168.1.11,GET /config.php HTTP/1.1 - 403
2,2025-03-29 12:13:00+00:00,cron,203.0.113.5,CRON job started: wget malicious.sh
3,2025-03-29 12:38:00+00:00,cron,203.0.113.5,CRON executed reverse_shell.py
4,2025-03-29 12:38:00+00:00,access,192.168.1.10,GET /admin HTTP/1.1 - 403


In [6]:
df['date'] = df['timestamp'].dt.date
df['time'] = df['timestamp'].dt.time

In [7]:
df.head()

,timestamp,log_type,client_ip,event,date,time
0,2025-03-29 12:19:00+00:00,auth,192.168.1.11,Accepted password for root from 192.168.1.11 p...,2025-03-29,12:19:00
1,2025-03-29 10:46:00+00:00,access,192.168.1.11,GET /config.php HTTP/1.1 - 403,2025-03-29,10:46:00
2,2025-03-29 12:13:00+00:00,cron,203.0.113.5,CRON job started: wget malicious.sh,2025-03-29,12:13:00
3,2025-03-29 12:38:00+00:00,cron,203.0.113.5,CRON executed reverse_shell.py,2025-03-29,12:38:00
4,2025-03-29 12:38:00+00:00,access,192.168.1.10,GET /admin HTTP/1.1 - 403,2025-03-29,12:38:00


# Checking Informations

In [ ]:
df['log_type'].unique()

array(['auth', 'access', 'cron'], dtype=object)

In [9]:
df['event'].unique()

array(['Accepted password for root from 192.168.1.11 port 54321 ssh2',
       'GET /config.php HTTP/1.1 - 403',
       'CRON job started: wget malicious.sh',
       'CRON executed reverse_shell.py', 'GET /admin HTTP/1.1 - 403',
       'POST /login.php HTTP/1.1 - 200',
       'new user: name=attacker, UID=1001, GID=1001',
       'Failed password for invalid user admin from 203.0.113.5 port 45678 ssh2',
       'CRON job added by attacker', 'GET /index.html HTTP/1.1 - 200',
       'Accepted password for root from 203.0.113.5 port 54321 ssh2',
       'Failed password for invalid user admin from 192.168.1.10 port 45678 ssh2',
       'Accepted password for root from 192.168.1.10 port 54321 ssh2',
       'Failed password for invalid user admin from 192.168.1.11 port 45678 ssh2',
       'GET /phpmyadmin/ HTTP/1.1 - 404',
       'Failed password for invalid user admin from 10.0.0.8 port 45678 ssh2'],
      dtype=object)

In [10]:
df['client_ip'].unique()

array(['192.168.1.11', '203.0.113.5', '192.168.1.10', '10.0.0.8'],
      dtype=object)

In [11]:
df['date'].nunique()

1

In [12]:
group_df = df.groupby(['date', 'client_ip']).agg({'event': 'count'}).reset_index()
group_df.columns = ['date', 'client_ip', 'event_count']

In [13]:
group_df

,date,client_ip,event_count
0,2025-03-29,10.0.0.8,21
1,2025-03-29,192.168.1.10,30
2,2025-03-29,192.168.1.11,22
3,2025-03-29,203.0.113.5,27


# Overall Logs

In [38]:
df.sort_values(by='timestamp')

,timestamp,log_type,client_ip,event,date,time
23,2025-03-29 10:00:00+00:00,cron,192.168.1.10,CRON job started: wget malicious.sh,2025-03-29,10:00:00
21,2025-03-29 10:01:00+00:00,auth,192.168.1.10,Failed password for invalid user admin from 19...,2025-03-29,10:01:00
86,2025-03-29 10:01:00+00:00,auth,192.168.1.10,"new user: name=attacker, UID=1001, GID=1001",2025-03-29,10:01:00
54,2025-03-29 10:02:00+00:00,cron,192.168.1.10,CRON job added by attacker,2025-03-29,10:02:00
33,2025-03-29 10:02:00+00:00,cron,203.0.113.5,CRON executed reverse_shell.py,2025-03-29,10:02:00
...,...,...,...,...,...,...
50,2025-03-29 12:57:00+00:00,auth,192.168.1.10,Failed password for invalid user admin from 19...,2025-03-29,12:57:00
97,2025-03-29 12:57:00+00:00,cron,203.0.113.5,CRON job added by attacker,2025-03-29,12:57:00
56,2025-03-29 13:00:00+00:00,access,192.168.1.10,GET /index.html HTTP/1.1 - 200,2025-03-29,13:00:00
82,2025-03-29 13:00:00+00:00,auth,192.168.1.11,"new user: name=attacker, UID=1001, GID=1001",2025-03-29,13:00:00


In [62]:
df1 = df[df['client_ip'] == '192.168.1.11'].copy()
df2 = df[df['client_ip'] == '203.0.113.5'].copy()
df3 = df[df['client_ip'] == '192.168.1.10'].copy()
df4 = df[df['client_ip'] == '10.0.0.8'].copy()

# IP 192.168.1.11 Logs

In [63]:
df1.sort_values(by='timestamp')

,timestamp,log_type,client_ip,event,date,time
90,2025-03-29 10:04:00+00:00,access,192.168.1.11,GET /phpmyadmin/ HTTP/1.1 - 404,2025-03-29,10:04:00
47,2025-03-29 10:06:00+00:00,auth,192.168.1.11,"new user: name=attacker, UID=1001, GID=1001",2025-03-29,10:06:00
19,2025-03-29 10:18:00+00:00,access,192.168.1.11,GET /config.php HTTP/1.1 - 403,2025-03-29,10:18:00
96,2025-03-29 10:22:00+00:00,cron,192.168.1.11,CRON job started: wget malicious.sh,2025-03-29,10:22:00
77,2025-03-29 10:27:00+00:00,cron,192.168.1.11,CRON job started: wget malicious.sh,2025-03-29,10:27:00
38,2025-03-29 10:34:00+00:00,cron,192.168.1.11,CRON executed reverse_shell.py,2025-03-29,10:34:00
1,2025-03-29 10:46:00+00:00,access,192.168.1.11,GET /config.php HTTP/1.1 - 403,2025-03-29,10:46:00
83,2025-03-29 10:48:00+00:00,access,192.168.1.11,GET /index.html HTTP/1.1 - 200,2025-03-29,10:48:00
12,2025-03-29 10:59:00+00:00,access,192.168.1.11,GET /index.html HTTP/1.1 - 200,2025-03-29,10:59:00
59,2025-03-29 11:01:00+00:00,auth,192.168.1.11,Accepted password for root from 192.168.1.11 p...,2025-03-29,11:01:00


In [46]:
df1[df1['log_type'] == 'auth']['event'].to_list()

['Accepted password for root from 192.168.1.11 port 54321 ssh2',
 'Failed password for invalid user admin from 192.168.1.11 port 45678 ssh2',
 'Failed password for invalid user admin from 192.168.1.11 port 45678 ssh2',
 'Accepted password for root from 192.168.1.11 port 54321 ssh2',
 'new user: name=attacker, UID=1001, GID=1001',
 'Accepted password for root from 192.168.1.11 port 54321 ssh2',
 'new user: name=attacker, UID=1001, GID=1001']

In [47]:
df1[df1['log_type'] == 'access']['event'].to_list()

['GET /config.php HTTP/1.1 - 403',
 'GET /index.html HTTP/1.1 - 200',
 'GET /config.php HTTP/1.1 - 403',
 'GET /admin HTTP/1.1 - 403',
 'GET /phpmyadmin/ HTTP/1.1 - 404',
 'GET /index.html HTTP/1.1 - 200',
 'GET /phpmyadmin/ HTTP/1.1 - 404']

In [48]:
df1[df1['log_type'] == 'cron']['event'].to_list()

['CRON job added by attacker',
 'CRON executed reverse_shell.py',
 'CRON job added by attacker',
 'CRON executed reverse_shell.py',
 'CRON job started: wget malicious.sh',
 'CRON job started: wget malicious.sh',
 'CRON executed reverse_shell.py',
 'CRON job started: wget malicious.sh']

# IP 203.0.113.5 Logs

In [49]:
df2.sort_values(by='timestamp')

,timestamp,log_type,client_ip,event,date,time
33,2025-03-29 10:02:00+00:00,cron,203.0.113.5,CRON executed reverse_shell.py,2025-03-29,10:02:00
31,2025-03-29 10:19:00+00:00,access,203.0.113.5,GET /config.php HTTP/1.1 - 403,2025-03-29,10:19:00
11,2025-03-29 10:29:00+00:00,cron,203.0.113.5,CRON job added by attacker,2025-03-29,10:29:00
10,2025-03-29 10:35:00+00:00,access,203.0.113.5,POST /login.php HTTP/1.1 - 200,2025-03-29,10:35:00
9,2025-03-29 10:41:00+00:00,auth,203.0.113.5,Failed password for invalid user admin from 20...,2025-03-29,10:41:00
44,2025-03-29 10:42:00+00:00,auth,203.0.113.5,Failed password for invalid user admin from 20...,2025-03-29,10:42:00
65,2025-03-29 10:42:00+00:00,cron,203.0.113.5,CRON job added by attacker,2025-03-29,10:42:00
16,2025-03-29 10:47:00+00:00,auth,203.0.113.5,Accepted password for root from 203.0.113.5 po...,2025-03-29,10:47:00
85,2025-03-29 10:57:00+00:00,auth,203.0.113.5,Failed password for invalid user admin from 20...,2025-03-29,10:57:00
28,2025-03-29 10:57:00+00:00,access,203.0.113.5,POST /login.php HTTP/1.1 - 200,2025-03-29,10:57:00


In [50]:
df2[df2['log_type'] == 'auth']['event'].to_list()

['Failed password for invalid user admin from 203.0.113.5 port 45678 ssh2',
 'Accepted password for root from 203.0.113.5 port 54321 ssh2',
 'new user: name=attacker, UID=1001, GID=1001',
 'Failed password for invalid user admin from 203.0.113.5 port 45678 ssh2',
 'new user: name=attacker, UID=1001, GID=1001',
 'Accepted password for root from 203.0.113.5 port 54321 ssh2',
 'Failed password for invalid user admin from 203.0.113.5 port 45678 ssh2',
 'Failed password for invalid user admin from 203.0.113.5 port 45678 ssh2',
 'Failed password for invalid user admin from 203.0.113.5 port 45678 ssh2',
 'Accepted password for root from 203.0.113.5 port 54321 ssh2']

In [51]:
df2[df2['log_type'] == 'access']['event'].to_list()

['GET /admin HTTP/1.1 - 403',
 'POST /login.php HTTP/1.1 - 200',
 'POST /login.php HTTP/1.1 - 200',
 'GET /config.php HTTP/1.1 - 403',
 'GET /admin HTTP/1.1 - 403',
 'GET /admin HTTP/1.1 - 403',
 'GET /admin HTTP/1.1 - 403']

In [52]:
df2[df2['log_type'] == 'cron']['event'].to_list()

['CRON job started: wget malicious.sh',
 'CRON executed reverse_shell.py',
 'CRON job added by attacker',
 'CRON executed reverse_shell.py',
 'CRON job started: wget malicious.sh',
 'CRON job added by attacker',
 'CRON job started: wget malicious.sh',
 'CRON executed reverse_shell.py',
 'CRON executed reverse_shell.py',
 'CRON job added by attacker']

# IP 192.168.1.10 Logs

In [53]:
df3.sort_values(by='timestamp')

,timestamp,log_type,client_ip,event,date,time
23,2025-03-29 10:00:00+00:00,cron,192.168.1.10,CRON job started: wget malicious.sh,2025-03-29,10:00:00
86,2025-03-29 10:01:00+00:00,auth,192.168.1.10,"new user: name=attacker, UID=1001, GID=1001",2025-03-29,10:01:00
21,2025-03-29 10:01:00+00:00,auth,192.168.1.10,Failed password for invalid user admin from 19...,2025-03-29,10:01:00
54,2025-03-29 10:02:00+00:00,cron,192.168.1.10,CRON job added by attacker,2025-03-29,10:02:00
6,2025-03-29 10:19:00+00:00,access,192.168.1.10,GET /config.php HTTP/1.1 - 403,2025-03-29,10:19:00
29,2025-03-29 10:24:00+00:00,auth,192.168.1.10,"new user: name=attacker, UID=1001, GID=1001",2025-03-29,10:24:00
39,2025-03-29 10:32:00+00:00,access,192.168.1.10,GET /index.html HTTP/1.1 - 200,2025-03-29,10:32:00
72,2025-03-29 10:37:00+00:00,cron,192.168.1.10,CRON job started: wget malicious.sh,2025-03-29,10:37:00
48,2025-03-29 10:58:00+00:00,auth,192.168.1.10,"new user: name=attacker, UID=1001, GID=1001",2025-03-29,10:58:00
68,2025-03-29 10:59:00+00:00,access,192.168.1.10,GET /admin HTTP/1.1 - 403,2025-03-29,10:59:00


In [54]:
df3[df3['log_type'] == 'auth']['event'].to_list()

['Failed password for invalid user admin from 192.168.1.10 port 45678 ssh2',
 'Failed password for invalid user admin from 192.168.1.10 port 45678 ssh2',
 'Accepted password for root from 192.168.1.10 port 54321 ssh2',
 'new user: name=attacker, UID=1001, GID=1001',
 'Accepted password for root from 192.168.1.10 port 54321 ssh2',
 'new user: name=attacker, UID=1001, GID=1001',
 'Failed password for invalid user admin from 192.168.1.10 port 45678 ssh2',
 'new user: name=attacker, UID=1001, GID=1001',
 'new user: name=attacker, UID=1001, GID=1001',
 'Failed password for invalid user admin from 192.168.1.10 port 45678 ssh2',
 'new user: name=attacker, UID=1001, GID=1001']

In [55]:
df3[df3['log_type'] == 'access']['event'].to_list()

['GET /admin HTTP/1.1 - 403',
 'GET /config.php HTTP/1.1 - 403',
 'GET /config.php HTTP/1.1 - 403',
 'GET /index.html HTTP/1.1 - 200',
 'GET /phpmyadmin/ HTTP/1.1 - 404',
 'GET /index.html HTTP/1.1 - 200',
 'GET /phpmyadmin/ HTTP/1.1 - 404',
 'GET /admin HTTP/1.1 - 403',
 'GET /admin HTTP/1.1 - 403',
 'POST /login.php HTTP/1.1 - 200',
 'POST /login.php HTTP/1.1 - 200',
 'GET /config.php HTTP/1.1 - 403',
 'POST /login.php HTTP/1.1 - 200']

In [56]:
df3[df3['log_type'] == 'cron']['event'].to_list()

['CRON job added by attacker',
 'CRON job started: wget malicious.sh',
 'CRON job started: wget malicious.sh',
 'CRON job added by attacker',
 'CRON executed reverse_shell.py',
 'CRON job started: wget malicious.sh']

# IP 10.0.0.8 Logs

In [57]:
df4.sort_values(by='timestamp')

,timestamp,log_type,client_ip,event,date,time
76,2025-03-29 10:05:00+00:00,cron,10.0.0.8,CRON job started: wget malicious.sh,2025-03-29,10:05:00
37,2025-03-29 10:10:00+00:00,access,10.0.0.8,POST /login.php HTTP/1.1 - 200,2025-03-29,10:10:00
36,2025-03-29 10:14:00+00:00,cron,10.0.0.8,CRON job added by attacker,2025-03-29,10:14:00
43,2025-03-29 10:15:00+00:00,cron,10.0.0.8,CRON executed reverse_shell.py,2025-03-29,10:15:00
14,2025-03-29 10:20:00+00:00,access,10.0.0.8,GET /admin HTTP/1.1 - 403,2025-03-29,10:20:00
99,2025-03-29 10:24:00+00:00,auth,10.0.0.8,Failed password for invalid user admin from 10...,2025-03-29,10:24:00
30,2025-03-29 10:53:00+00:00,access,10.0.0.8,GET /phpmyadmin/ HTTP/1.1 - 404,2025-03-29,10:53:00
26,2025-03-29 10:59:00+00:00,cron,10.0.0.8,CRON job started: wget malicious.sh,2025-03-29,10:59:00
8,2025-03-29 11:00:00+00:00,auth,10.0.0.8,"new user: name=attacker, UID=1001, GID=1001",2025-03-29,11:00:00
80,2025-03-29 11:03:00+00:00,access,10.0.0.8,GET /phpmyadmin/ HTTP/1.1 - 404,2025-03-29,11:03:00


In [58]:
df4[df4['log_type'] == 'auth']['event'].to_list()

['new user: name=attacker, UID=1001, GID=1001',
 'Failed password for invalid user admin from 10.0.0.8 port 45678 ssh2',
 'Failed password for invalid user admin from 10.0.0.8 port 45678 ssh2',
 'new user: name=attacker, UID=1001, GID=1001',
 'new user: name=attacker, UID=1001, GID=1001',
 'Failed password for invalid user admin from 10.0.0.8 port 45678 ssh2']

In [59]:
df4[df4['log_type'] == 'access']['event'].to_list()

['POST /login.php HTTP/1.1 - 200',
 'GET /admin HTTP/1.1 - 403',
 'GET /phpmyadmin/ HTTP/1.1 - 404',
 'POST /login.php HTTP/1.1 - 200',
 'POST /login.php HTTP/1.1 - 200',
 'GET /phpmyadmin/ HTTP/1.1 - 404',
 'GET /admin HTTP/1.1 - 403',
 'GET /index.html HTTP/1.1 - 200']

In [60]:
df4[df4['log_type'] == 'cron']['event'].to_list()

['CRON job started: wget malicious.sh',
 'CRON job added by attacker',
 'CRON executed reverse_shell.py',
 'CRON executed reverse_shell.py',
 'CRON job added by attacker',
 'CRON job added by attacker',
 'CRON job started: wget malicious.sh']